### Setup and Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from neural_final_proj.core_models import CustomCNN, ResNetSmall 
from neural_final_proj.project_utils import prepare_data_loaders, evaluate_model
from neural_final_proj.adversarial_logic import generate_adversarial_set, train_robust_model

class ConfigC:
    """
    Configuration for Problem C: Adversarial Training (PGD-AT).
    Uses attack parameters determined in Problem B.
    """
    data_root = "data/Sports"
    image_size = 64
    batch_size = 64
    num_classes = 10 
    num_epochs = 20
    device = torch.device("cpu")
    

    out_dir = "problem_C_outputs" 
    ckpt_dir = "problem_A_outputs" 
    model_A_ckpt = "CustomCNN-original.pt"
    model_to_robustify = "CustomCNN"
    
    # --- Adversarial Training (AT) Settings ---
    # These parameters must match the strongest attack from Problem B
    PGD_EPS = 8 / 255.0
    PGD_ALPHA = 2 / 255.0
    PGD_STEPS = 20

cfg = ConfigC()

In [3]:
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir('..')
current_folder_name = os.path.basename(os.getcwd())

# Create output directory
os.makedirs(cfg.out_dir, exist_ok=True)

WIDTH = 20
print(f"{'Current directory':<{WIDTH}}: {current_folder_name}")
print(f"{'Model to Robustify':<{WIDTH}}: {cfg.model_to_robustify}")
print(f"{'Device':<{WIDTH}}: {cfg.device}")
print(f"{'PGD Epsilon (L-inf)':<{WIDTH}}: {cfg.PGD_EPS:.4f}")
print(f"{'Output Location':<{WIDTH}}: {cfg.out_dir}\n")

Current directory   : EE4745_Final_Proj
Model to Robustify  : CustomCNN
Device              : cpu
PGD Epsilon (L-inf) : 0.0314
Output Location     : problem_C_outputs



### Execute Training and Evaluate Robustness

In [4]:
# Load Data Loaders (using helper from project_utils)
train_loader, val_loader, class_names, _ = prepare_data_loaders(
    cfg.data_root, cfg.image_size, cfg.batch_size
)

# Select Model to Train
model_class = CustomCNN
model_name = "CustomCNN_Robust"
model_robust = model_class(num_classes=cfg.num_classes).to(cfg.device)

# Start Adversarial Training
results_robust_cnn = train_robust_model(model_robust, model_name, train_loader, val_loader, cfg)

print(f"\nCustomCNN Training Complete.")
WIDTH = 26
print(f"{'Final Validation Accuracy':<{WIDTH}}: {results_robust_cnn['final_val_accuracy']:.2f}%")
print(f"{'Total Time Taken':<{WIDTH}}: {results_robust_cnn['total_training_time']:.2f}s")


--- Starting ADVERSARIAL TRAINING for CustomCNN_Robust ---
Epoch 1/20 | Train Acc: 24.61% | Val Acc: 38.00% | Time: 23.42s
Epoch 2/20 | Train Acc: 37.73% | Val Acc: 46.00% | Time: 22.68s
Epoch 3/20 | Train Acc: 43.94% | Val Acc: 46.00% | Time: 23.41s
Epoch 4/20 | Train Acc: 46.70% | Val Acc: 48.00% | Time: 22.69s
Epoch 5/20 | Train Acc: 50.91% | Val Acc: 56.00% | Time: 23.51s
Epoch 6/20 | Train Acc: 56.06% | Val Acc: 58.00% | Time: 22.61s
Epoch 7/20 | Train Acc: 58.76% | Val Acc: 62.00% | Time: 23.02s
Epoch 8/20 | Train Acc: 62.84% | Val Acc: 58.00% | Time: 22.25s
Epoch 9/20 | Train Acc: 65.10% | Val Acc: 66.00% | Time: 22.37s
Epoch 10/20 | Train Acc: 68.49% | Val Acc: 56.00% | Time: 22.41s
Epoch 11/20 | Train Acc: 71.44% | Val Acc: 58.00% | Time: 21.80s
Epoch 12/20 | Train Acc: 73.70% | Val Acc: 60.00% | Time: 22.23s
Epoch 13/20 | Train Acc: 76.02% | Val Acc: 64.00% | Time: 22.54s
Epoch 14/20 | Train Acc: 79.60% | Val Acc: 64.00% | Time: 22.54s
Epoch 15/20 | Train Acc: 83.36% | Val A

In [15]:
# Load Original Model for Comparison
model_original = model_class(num_classes=cfg.num_classes).to(cfg.device)
original_ckpt_path = os.path.join(cfg.ckpt_dir, cfg.model_A_ckpt)
print(f"Loading original model from      : {original_ckpt_path}")
model_original.load_state_dict(torch.load(original_ckpt_path, map_location=cfg.device))

# Reload Best Robust Model Checkpoint
robust_ckpt_path = os.path.join(cfg.out_dir, f"{results_robust_cnn['model_name']}-robust.pt")
print(f"Loading best robust weights from : {robust_ckpt_path}")
model_robust.load_state_dict(torch.load(robust_ckpt_path, map_location=cfg.device))

Loading original model from      : problem_A_outputs\CustomCNN-original.pt
Loading best robust weights from : problem_C_outputs\CustomCNN_Robust-robust.pt


<All keys matched successfully>

### Evaluation

In [ ]:
# 1. Clean Accuracy (Utility Test)
_, clean_acc_robust, _, _ = evaluate_model(model_robust, val_loader, nn.CrossEntropyLoss(), cfg.device)
_, clean_acc_original, _, _ = evaluate_model(model_original, val_loader, nn.CrossEntropyLoss(), cfg.device)

# Generate PGD attacks on the validation set using the ROBUST model
adv_robust_set = generate_adversarial_set(
    model_robust, val_loader, 'pgd', cfg.PGD_EPS, targeted=False, target_class_idx=-1,
    num_samples=len(val_loader.dataset), alpha=cfg.PGD_ALPHA, steps=cfg.PGD_STEPS
)

# Robust Accuracy = 100 * (1 - Attack Success Rate)
robust_acc_robust = (len(adv_robust_set) - sum(int(e["success"]) for e in adv_robust_set)) / len(adv_robust_set) * 100

# Generate PGD attacks on the validation set using the ORIGINAL model
adv_original_set = generate_adversarial_set(
    model_original, val_loader, 'pgd', cfg.PGD_EPS, targeted=False, target_class_idx=-1,
    num_samples=len(val_loader.dataset), alpha=cfg.PGD_ALPHA, steps=cfg.PGD_STEPS
)
robust_acc_original = (len(adv_original_set) - sum(int(e["success"]) for e in adv_original_set)) / len(adv_original_set) * 100

### Final Reporting

In [23]:
WIDTH_MODEL = 20
WIDTH_ACCURACY = 25
WIDTH_ROBUST_ACC = 25

print(f"\nAdversarial Training Parameters: PGD Epsilon={cfg.PGD_EPS:.4f}, Steps={cfg.PGD_STEPS}")
print("\n"+5*" "+"Adversarial Results\n"+(WIDTH_MODEL + WIDTH_ACCURACY + WIDTH_ROBUST_ACC)*"-")

print(
    f"{'Model':<{WIDTH_MODEL}}" +
    f"{'Clean Accuracy (%)':<{WIDTH_ACCURACY}}" +
    f"{'Robust Accuracy (PGD %)':<{WIDTH_ROBUST_ACC}}"
)

results_data = [
    {"Model": "Original Model", "Clean Acc": clean_acc_original, "Robust Acc": robust_acc_original},
    {"Model": "Robust Model", "Clean Acc": clean_acc_robust, "Robust Acc": robust_acc_robust}
]

for r in results_data:
    print(
        f"{r['Model']:<{WIDTH_MODEL}}" +
        f"{r['Clean Acc']:<{WIDTH_ACCURACY}.2f}" + # Using .2f for two decimal places
        f"{r['Robust Acc']:<{WIDTH_ROBUST_ACC}.2f}"
    )


Adversarial Training Parameters: PGD Epsilon=0.0314, Steps=20

     Adversarial Results
----------------------------------------------------------------------
Model               Clean Accuracy (%)       Robust Accuracy (PGD %)  
Original Model      80.00                    65.00                    
Robust Model        70.00                    85.71                    
